# Task in this notebook

**List of tasks**
* EDA (Some EDA)
* Image PreProcessing (like augmentation)
* Divided dataset into
* * Train Dataset
* * Validation Dataset
* Basic Modeling Without Pre-Trained and check Loss curve
* Add pre_trained Model and check loss curve
* Some Tuning (if possible)

# 1. Setting basic configurations

In [1]:
from IPython.display import clear_output
!pip install -qU scikit-learn
clear_output()

In [2]:
%config Completer.use_jedi = False
import os, re, math, random, shutil, warnings, gc
import glob
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import cv2

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
from matplotlib.ticker import StrMethodFormatter
from mpl_toolkits.axes_grid1 import ImageGrid
from kaggle_datasets import KaggleDatasets
from kaggle_secrets import UserSecretsClient

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None
tf.get_logger().setLevel('ERROR')
py.init_notebook_mode(connected=True)


/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.



In [ ]:
#if os.path.exists('/kaggle/working/Training_Images/Non_cancerous'):
    #os.remove('/kaggle/working/Training_Images/Non_cancerous')
   # print("File removed successfully")

In [3]:
devices = ['TPU', 'GPU']
img_size = [(1024,512), (512,256)]
img_ext = ['png']

class Config:
    
    def __init__(self):
        
        self.project_name="RSNA_BCD_Modeling"
        self.device=devices[0]
        self.seed=150
        
        self.path='/kaggle/input/rsna-bcs-imagepreprosessing/'
        
        self.training_images_path='/kaggle/working/Training_Images/'
        self.cancerous='Cancerous/'
        self.non_cancerous='Non_cancerous/'
        self.train_csv=self.path+'TrainDataset.csv'
        self.img_size = img_size[1]
        self.img_ext = img_ext[0]
        

config=Config()
        

# 2. Seeding Random Generators

In [4]:
def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    
seeding(config.seed)

# 3.Configur devices

In [ ]:
# if config.device == 'TPU':

#     print('Connecting to TPU...')
#     try:
#         tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
#     except ValueError:
#         raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

#     tf.keras.mixed_precision.set_global_policy("mixed_bfloat16")
#     tf.config.set_soft_device_placement(True)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)

#     #config.path = KaggleDatasets().get_gcs_path(config.path.split('/')[3])
#     #config.train_path = config.path + '/' + 'train_images/'
#     #config.num_devices = strategy.num_replicas_in_sync
#     #config.parameters['batch_size'] = config.parameters['batch_size'] * config.num_devices

#     #print(f'Running on {config.num_devices} TPU devices')


# if config.device == 'GPU':
    
#     num_devices = len(tf.config.list_physical_devices('GPU'))

#     if num_devices > 1:
#         config.num_devices = num_devices
#         strategy = tf.distribute.MirroredStrategy()
#         #config.parameters['batch_size'] = config.parameters['batch_size'] * config.num_devices
#         #print(f'Running on {num_devices} GPU devices')
#     elif num_devices == 1:
#         strategy = tf.distribute.get_strategy()
#         print(f'Running on {num_devices} GPU device')
#     else:
#         strategy = tf.distribute.get_strategy()
#         config.device = 'CPU'
#         print(f'Running on CPU')
    
#     tf.config.optimizer.set_jit(True)
#     tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [5]:
shutil.rmtree(config.training_images_path+config.non_cancerous)
shutil.rmtree(config.training_images_path+config.cancerous)

In [6]:
# Create the "Cancer" directory
cancer_dir = os.path.join(config.training_images_path, config.cancerous)
os.makedirs(os.path.dirname(cancer_dir),exist_ok=True)

In [7]:
# Create the "No Cancer" directory
no_cancer_dir = os.path.join(config.training_images_path,config.non_cancerous)
os.makedirs(os.path.dirname(no_cancer_dir),exist_ok=True)

In [5]:
train_df = pd.read_csv(config.train_csv)
train_df.head()

,Unnamed: 0,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,input_image_path,output_image_path
0,0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/462822612.png
1,1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1459541791.png
2,2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1864590858.png
3,3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10006/1874946579.png
4,4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,/kaggle/input/rsna-breast-cancer-detection/tra...,/kaggle/working/train_images/10011/220375232.png


In [6]:
test_image_dirs = glob.glob(config.path+"test_images/*/*.png")
train_image_dirs = glob.glob(config.path+"train_images/*/*.png")

images_Cancer=[]
images_noCancer=[]

print(train_df.shape)
i=0
for image in train_image_dirs:
    
    imageId=str(image.split('/')[6].split('.')[0])
    
  
    cancer_status=str(train_df.loc[train_df["image_id"].astype(str)==imageId].iloc[0]["cancer"])
    
    if(cancer_status=='1'):
        images_Cancer.append(imageId)
    else:
        images_noCancer.append(imageId)

print("Number of training images with Cancer:",len(images_Cancer))
print("number of training images with no cancer:",len(images_noCancer))
print("total number of training images : ",len(train_image_dirs))
print("Data type of image no cancer :",type(images_noCancer))

(54706, 17)
Number of training images with Cancer: 1158
number of training images with no cancer: 7465
total number of training images :  8623
Data type of image no cancer : <class 'list'>


In [9]:

images_noCancer_=images_noCancer[:2000]
print(len(images_noCancer_))

2000


# 4. Take 2000 images of non cancereous and all the cancereous images for model training

In [10]:
datagen = ImageDataGenerator()

# Get the filenames and labels of the images in the source directory
data = datagen.flow_from_directory(
        directory=config.path,
        batch_size=32,
        class_mode='binary',
        shuffle=False)


Found 8627 images belonging to 2 classes.


In [11]:
cancereous_images = []
non_cancereous_images=[]
for i in range(len(data.filenames)):
    
    if str(data.filenames[i].split('/')[0])=='train_images':
        
        if str(data.filenames[i].split('/')[2].split('.')[0]) in images_noCancer_:
            img = cv2.imread(str(config.path)+data.filenames[i])
            img=np.array(img)
            #np.divide(img, 255., out=img,casting='unsafe')
            non_cancereous_images.append(img)
            #shutil.copy(os.path.join(config.path,data.filenames[i]), no_cancer_dir)
        elif str(data.filenames[i].split('/')[2].split('.')[0]) in images_Cancer:
            img = cv2.imread(str(config.path)+data.filenames[i])
            img=np.array(img)
            #np.divide(img, 255., out=img,casting='unsafe')
            cancereous_images.append(img)
            #cancereous_images = np.array(cancereous_images) / 255. 
            #plt.imshow(cancereous_images[0])
            #shutil.copy(os.path.join(config.path,data.filenames[i]),cancer_dir)
        else:
            continue
            

In [24]:
print(len(cancereous_images))

1158


**Find the class to be augmented**

In [1]:
# Select the images belonging to the class to augment

cancereous_images = np.array(cancereous_images, )
cancereous_images /= 255.

non_cancereous_images=np.array(non_cancereous_images,dtype=np.float64)
non_cancereous_images/=255.


n_augment = len(non_cancereous_images)-len(cancereous_images)
idx_to_aug = np.random.randint(cancereous_images.shape[0], size=n_augment)
print("number of augumented images :",n_augment)


NameError: name 'np' is not defined

In [31]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ])

In [34]:
augumented_images = []

plt.figure(figsize=(10, 10))
for i in range(n_augment):
    augmented_image = data_augmentation(cancereous_images[idx_to_aug[i]])
    augumented_images.append(augmented_image)
    
        

<Figure size 1000x1000 with 0 Axes>

In [35]:
i=0
for aug_img in augumented_images:
    i +=1
    print(aug_img)
    
    if i ==2:
        break

tf.Tensor(
[[[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.12278712 0.12278712 0.12278712]
  [0.13759224 0.13759224 0.13759224]
  [0.16125666 0.16125666 0.16125666]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.13603817 0.13603817 0.13603817]
  [0.15399447 0.15399447 0.15399447]
  [0.17608501 0.17608501 0.17608501]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.13220704 0.13220704 0.13220704]
  [0.15024993 0.15024993 0.15024993]
  [0.16102941 0.16102941 0.16102941]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.76377857 0.76377857 0.76377857]
  [0.69553196 0.69553196 0.69553196]
  [0.6270425  0.6270425  0.6270425 ]]

 [[0.         0.         0.        ]
  [0.         0.         0

In [ ]:
train_df.info()

# 3. Exploratory Data Analysis (EDA)

In [ ]:
fig = go.Figure(data=[go.Pie(labels=['Non cancerous', 'Cancerous'], values=train_df['cancer'].value_counts(), hole=.3)])
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Number of patients with cancer', title_x=0.5)
fig.show()


In [ ]:
fig = px.histogram(train_df, x="age", title='Age distribution of the patients')
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:

fig = px.violin(train_df.sample(frac=0.1), y="age", x="cancer", color="cancer", box=True, 
                points="all",
                title='Age distribution with respect to the cancer')
fig.update_layout(title_x=0.5)
fig.show()


In [ ]:
fig = px.histogram(train_df.groupby('patient_id').count().reset_index(), x="image_id", color='image_id',
                   title='Number of images per patient')
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Laterality', 'Density', 'View', 'Implant'))

fig.add_trace(go.Bar(x=train_df['laterality'].value_counts().index, y=train_df['laterality'].value_counts().values), row=1, col=1)
fig.add_trace(go.Bar(x=train_df['density'].value_counts().index, y=train_df['density'].value_counts().values), row=1, col=2)
fig.add_trace(go.Bar(x=train_df['view'].value_counts().index, y=train_df['view'].value_counts().values), row=2, col=1)
fig.add_trace(go.Bar(x=train_df['implant'].value_counts().index, y=train_df['implant'].value_counts().values), row=2, col=2)

fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Count of laterality, density, view and implant', title_x=0.5)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=('Laterality', 'Density', 'View', 'Implant'))

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['laterality'].value_counts().index, y=train_df[train_df['cancer'] == 1]['laterality'].value_counts().values, name='Cancer'), row=1, col=1)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['laterality'].value_counts().index, y=train_df[train_df['cancer'] == 0]['laterality'].value_counts().values, name='No Cancer'), row=1, col=1)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['density'].value_counts().index, y=train_df[train_df['cancer'] == 1]['density'].value_counts().values, name='Cancer'), row=1, col=2)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['density'].value_counts().index, y=train_df[train_df['cancer'] == 0]['density'].value_counts().values, name='No Cancer'), row=1, col=2)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['view'].value_counts().index, y=train_df[train_df['cancer'] == 0]['view'].value_counts().values, name='No Cancer'), row=2, col=1)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['view'].value_counts().index, y=train_df[train_df['cancer'] == 1]['view'].value_counts().values, name='Cancer'), row=2, col=1)

fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 0]['implant'].value_counts().index, y=train_df[train_df['cancer'] == 0]['implant'].value_counts().values, name='No Cancer'), row=2, col=2)
fig.add_trace(go.Bar(x=train_df[train_df['cancer'] == 1]['implant'].value_counts().index, y=train_df[train_df['cancer'] == 1]['implant'].value_counts().values, name='Cancer'), row=2, col=2)

fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Count of laterality, density, view and implant with respect to the cancer', title_x=0.5)
fig.show()

# 4. Missing Values 

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percentage = (data.isnull().sum()/data.isnull().count()).sort_values(ascending = False)
    return pd.concat([total,percentage] , axis = 1 , keys = ['Total' , 'Percent'])

missing_data(train_df)

In [ ]:
train_df = train_df.drop(['BIRADS'], axis=1)
train_df['age'] = train_df['age'].fillna(train_df['age'].mean())
train_df['density'] = train_df['density'].fillna('E')

In [ ]:
missing_data(train_df)

In [ ]:
train_df['laterality'] = train_df['laterality'].astype('category')
train_df['view'] = train_df['view'].astype('category')
train_df['age'] = train_df['age'].astype('int64')
train_df['output_image_path'] = train_df['output_image_path'].astype('string')

# 5. Fix Data Types

In [ ]:
train_df.info()

# 6. Numerical Categorization

In [ ]:
train_df["age_bin"] = pd.cut(train_df['age'].values.reshape(-1), bins=5, labels=False)

# 7. Generate Dummy Variables 

In [ ]:
cat_cols = ['laterality', 'view', 'density', 'difficult_negative_case']
train_df = pd.get_dummies(train_df, columns=cat_cols)

In [ ]:
train_df.info()